Used to generate LGBM predictions

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, r2_score, mean_squared_error

from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from category_encoders import MEstimateEncoder, TargetEncoder
from sklearn.inspection import permutation_importance

import seaborn as sns
import shap

import lightgbm as lgb

import sys
sys.path.append('../lightgbm')

C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
C:\Users\Andrew\anaconda3\envs\ml_env\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def delta_minimization_order(al

In [2]:
process_train_path = 'processed/processed_train_2.csv'
test_df_path = '../../Dataset/test.csv/test.csv'
processed_test_path = 'processed/processed_test_2.csv'

In [3]:
processed_df = pd.read_csv(process_train_path)
X = processed_df.drop(['price_doc'], axis=1)
y = processed_df['price_doc']

In [4]:
# Best for now
params2 = {
    'boosting': 'gbdt', 
    'lambda_l2': 7.068023653932577, 
    'bagging_fraction': 0.6261319161311689, 
    'num_leaves': 151, 
    'feature_fraction': 0.976804739696195, 
    'max_depth': 8,
    'learning_rate': 0.01552248392804131, 
    'subsample': 0.4408875142865268, 
    'colsample_bytree': 0.3829262582959811, 
    'min_data_in_leaf': 21,
    "objective": "regression",
    "metric": "rmse",
    "n_estimators": 825,
    "verbosity": -1,
}

# Best for now
params4 = {
    'boosting': 'gbdt', 
    'lambda_l2': 7.068023653932577, 
    'bagging_fraction': 0.6261319161311689, 
    'num_leaves': 151, 
    'feature_fraction': 0.976804739696195, 
    'max_depth': 8,
    'learning_rate': 0.01552248392804131, 
    'subsample': 0.4408875142865268, 
    'colsample_bytree': 0.3829262582959811, 
    'min_data_in_leaf': 21,
    "objective": "regression",
    "metric": "rmse",
    "n_estimators": 825,
    "verbosity": -1,
}

params3 = {
    'n_estimators': 957,
     'lambda_l2': 1.0179522290678555,
     'bagging_fraction': 0.803425892987424,
     'num_leaves': 730,
     'feature_fraction': 0.756874719068369,
     'max_depth': 9,
     'learning_rate': 0.010123433800404112,
     'subsample': 0.4105167842639978,
     'colsample_bytree': 0.5230497041584328,
     'min_data_in_leaf': 10,
     'objective': 'regression',
     'metric': 'rmse',
     'verbosity': -1
}

model = lgb.LGBMRegressor(**params4)

model.fit(X, y)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.976804739696195, colsample_bytree=0.3829262582959811 will be ignored. Current value: feature_fraction=0.976804739696195
[LightGBM] [Warning] min_data_in_leaf is set=21, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=21
[LightGBM] [Warning] bagging_fraction is set=0.6261319161311689, subsample=0.4408875142865268 will be ignored. Current value: bagging_fraction=0.6261319161311689
[LightGBM] [Warning] lambda_l2 is set=7.068023653932577, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.068023653932577


LGBMRegressor(bagging_fraction=0.6261319161311689, boosting='gbdt',
              colsample_bytree=0.3829262582959811,
              feature_fraction=0.976804739696195, lambda_l2=7.068023653932577,
              learning_rate=0.01552248392804131, max_depth=8, metric='rmse',
              min_data_in_leaf=21, n_estimators=825, num_leaves=151,
              objective='regression', subsample=0.4408875142865268,
              verbosity=-1)

In [5]:
test_df = pd.read_csv(test_df_path)
processed_test_df = pd.read_csv(processed_test_path)
processed_test_df = processed_test_df.drop(['id'], axis=1)
processed_test_df

,full_sq,life_sq,floor,max_floor,build_year,num_room,kitch_sq,state,area_m,raion_popul,...,big_road1_1line_yes,railroad_1line_no,railroad_1line_yes,material_1.0,material_2.0,material_3.0,material_4.0,material_5.0,material_6.0,material_nan
0,39.00,20.700000,2,9,1998.000000,1,8.9,3.000000,2.615514e+07,178264,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,79.20,34.404467,8,17,0.000000,3,1.0,1.000000,2.553630e+07,4001,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40.50,25.100000,3,5,1960.000000,2,4.8,2.000000,9.946335e+06,139322,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,62.80,36.000000,17,17,2016.000000,2,62.8,3.000000,2.149409e+07,7122,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40.00,40.000000,17,17,0.000000,1,1.0,1.000000,2.553630e+07,4001,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,52.20,31.800000,10,12,1973.000000,2,9.1,2.000000,7.811375e+06,153248,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7658,54.09,34.404467,14,0,1879.046638,2,0.0,2.105145,5.299528e+07,13890,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7659,41.08,1.000000,12,1,1.000000,1,1.0,1.000000,7.307411e+06,75377,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7660,34.80,19.800000,8,9,1977.000000,1,6.4,2.000000,7.128794e+06,145576,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
pred = model.predict(processed_test_df)

# pred = np.expm1(pred)

prediction_df = pd.DataFrame({
    'id': test_df['id'],
    'price_doc': pred
}).to_csv('lgbm_pred.csv', index=False)